In [7]:
# Import all relevant libraries
import pandas as pd
import numpy as np
import functools, collections
from tqdm.notebook import tqdm
tqdm.pandas()
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Create the SLA matrix based on SLA Matrix.xlsx
d = {'metro manila': [3,5,7,7],
     'luzon':        [5,5,7,7],
     'visayas':      [7,7,7,7],
     'mindanao':     [7,7,7,7]}
index_row = ['metro manila', 'luzon', 'visayas', 'mindanao']
SLA_matrix = pd.DataFrame(data=d, index=index_row)

In [3]:
# Read in the delivery orders data and put it into a dataframe
df = pd.read_csv('delivery_orders_march.csv')

In [4]:
# Transform buyer and seller address to simplified area
def simplify_add(list_of_add):
    for word in index_row:
        if word in list_of_add.lower():
            return(word)
        
df['buy_add'] = df['buyeraddress'].apply(simplify_add)
df['sell_add'] = df['selleraddress'].apply(simplify_add)

In [5]:
def get_SLA(buy_add, sell_add):
    SLA = SLA_matrix.loc[buy_add][sell_add]
    return SLA

# Get corresponding SLA for each row
df['first_attempt'] = df[['buy_add','sell_add']].progress_apply(lambda x: get_SLA(*x), axis=1)

In [8]:
def change_time(time_list):
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time_list))
    return new_time

# Change the time format
df['pick'] = df['pick'].apply(change_time)
df['1st_deliver_attempt'] = df['1st_deliver_attempt'].apply(change_time)
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].fillna(0)
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].apply(change_time)

In [9]:
# Change fields into datatime format
df['pick'] = pd.to_datetime(df['pick'])
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'])
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'])

In [10]:
# Get the date of the fields only
df['pick_date_only'] = df['pick'].dt.date
df['1st_deliver_attempt_date'] = df['1st_deliver_attempt'].dt.date
df['2nd_deliver_attempt_date'] = df['2nd_deliver_attempt'].dt.date

In [11]:
# Get the number of days between pickup and 1st delivery attempt
df['1st_diff'] = df['1st_deliver_attempt_date'] - df['pick_date_only']
df['1st_diff'] = df['1st_diff'].dt.days

In [12]:
# This function returns the number of working days between two dates
def num_working_days(start,end):
    valid_calendar = np.busdaycalendar(weekmask='1111110', holidays=['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])
    valid_daycount = np.busday_count(start, end, busdaycal=valid_calendar)
    return valid_daycount

In [13]:
# Rename the columns for better clarity
df.rename(columns={'1st_deliver_attempt_date': 'first_attempt_date',
                  '2nd_deliver_attempt_date': 'second_attempt_date'}, inplace=True)

In [15]:
# Get the number of working days between pickup and first attempt + first attempt and second attempt
df['actual_day_used'] = num_working_days(df['pick_date_only'], df['first_attempt_date'])
df['second_actual_day_used'] = num_working_days(df['first_attempt_date'], df['second_attempt_date'])

In [16]:
# Check if actual days taken exceeds the allowed number of days
df['first_is_late'] = df['actual_day_used'] > df['first_attempt']
df['second_is_late'] = df['second_actual_day_used'] > 3
df['is_late'] = (df['second_is_late']) | (df['first_is_late'])

In [17]:
# Select the columns 'orderid' and 'is_late' for submission
df_output = df[['orderid','is_late']]
df_output['is_late'] = df_output['is_late'].astype(int)

In [18]:
# Save the results as a csv file
df_output.to_csv('submission.csv', index=False, header=True)